<a href="https://colab.research.google.com/github/ohoelske/Databases/blob/main/210116_Kursaufgabe_Woche_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Kursaufgabe Woche 2 - COVID-19 Patients**

# Importing libraries

In [19]:
import pandas as pd
import numpy as np
import sqlite3 as sq
from sqlite3 import Error
from google.colab import drive

In [20]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [21]:
material_path = "/content/drive/MyDrive/csv_data_Oliver"

In [22]:
# list of diseases
!ls "/content/drive/MyDrive/csv_data_Oliver"

 allergy	 colorectal_cancer   lung_cancer
 asthma		 covid19	     metabolic_syndrome_disease
 breast_cancer	 dermatitis	    'Outline COVID-19.docx'


# Path of input/output data

In [23]:
## type of patients
patient_type = "covid19"

In [24]:
# source database path
DB_FILE_PATH = f"{material_path}/source_dbs/source_{patient_type}_test.db"

# folder contains csv files
csv_path = f"{material_path}/csv_data/{patient_type}/"

DB_FILE_PATH, csv_path

('/content/drive/MyDrive/csv_data_Oliver/source_dbs/source_covid19_test.db',
 '/content/drive/MyDrive/csv_data_Oliver/csv_data/covid19/')

#SQL queries to create tables# 

In [25]:
# dictionary key: name of table, values: sql query to create table
sql_table = {}

In [26]:
# sql query to create disease table
sql_table["disease"] = """CREATE TABLE IF NOT EXISTS disease (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)
                           
                       );"""

In [27]:
#  sql query to create patients table
sql_table["patients"] = """CREATE TABLE IF NOT EXISTS patients (
                           Id STRING PRIMARY KEY,
                           BIRTHDATE DATE,
                           DEATHDATE DATE,
                           SSN STRING,
                           DRIVERS STRING,
                           PASSPORT STRING,
                           PREFIX STRING,
                           FIRST STRING,
                           LAST STRING,
                           SUFFIX STRING,
                           MAIDEN STRING,
                           MARITAL STRING,
                           RACE STRING,
                           ETHNICITY STRING,
                           GENDER STRING,
                           BIRTHPLACE STRING,
                           ADDRESS STRING,
                           CITY STRING,
                           STATE STRING,
                           COUNTRY STRING,
                           ZIP STRING,
                           LAT INTEGER,
                           LON INTEGER,
                           HEALTHCARE_EXPENSES INTEGER,
                           HEALTHCARE_COVERAGE INTEGER
                       );"""
      

In [28]:
# sql query to create encounters table
sql_table["encounters"] = """CREATE TABLE IF NOT EXISTS encounters (
                           Id STRING PRIMARY KEY,
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ORGANIZATIONS STRING,
                           PROVIDER STRING,
                           PAYER STRING,
                           ENCOUNTERCLASS STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_ENCOUNTER_COST INTEGER,
                           TOTAL_CLAIM_COST INTEGER,
                           PAYER_COVERAGE INTEGER,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                       );"""


In [29]:
# sql query to create careplans table
sql_table["careplans"] = """CREATE TABLE IF NOT EXISTS careplans (
                           Id STRING PRIMARY KEY,
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""

In [30]:
# sql query to create conditions table
sql_table["conditions"] = """CREATE TABLE IF NOT EXISTS conditions (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""

In [31]:
# sql query to create medications table
sql_table["medications"] = """CREATE TABLE IF NOT EXISTS medications (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           PAYER STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST INTEGER,
                           PAYER_COVERAGE INTEGER,
                           DISPENSES INTEGER,
                           TOTALCOST INTEGER,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           
                       );"""


In [32]:
# # sql query to create procedures table
sql_table["procedures"]= """CREATE TABLE IF NOT EXISTS procedures (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST INTEGER,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""


In [33]:
# # sql query to create observations table
sql_table["observations"]= """CREATE TABLE IF NOT EXISTS observations (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           VALUE STRING,
                           UNITS STRING,
                           TYPE STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""

In [34]:
# sql query to create devices table
sql_table["devices"] = """CREATE TABLE IF NOT EXISTS devices (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           UDI STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)
                           
                       );"""


In [35]:
# sql query to create Imaging_studies table
sql_table["imaging_studies"] = """CREATE TABLE IF NOT EXISTS imaging_studies (
                           Id STRING PRIMARY KEY,
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           BODYSITE_CODE STRING,
                           BODYSITE_DESCRIPTION STRING,
                           MODALITY_CODE STRING,
                           MODALITY_DESCRIPTION STRING,
                           SOP_CODE STRING,
                           SOP_DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""

In [36]:
# sql query to create Immunizations table
sql_table["immunizations"]= """CREATE TABLE IF NOT EXISTS immunizations(
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST INTEGER,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""

# Import csv files into database tables

In [37]:
def create_connection_file():
    """ create a database connection to a database that resides
        in a file 'covid19_test.db'
    """
    conn = None;
    try:
       #Establishing the connection
        conn = sq.connect('/content/drive/MyDrive/csv_data_Oliver/covid19/covid19_test.db')
        return conn
        print(sq.version)
    except Error as e:
        print("Error while connecting to sqlite", e)

# Creating a temporary empty database in the memory.
conn = create_connection_file()
# Creating a cursor object using the cursor() method
cur = conn.cursor()

In [38]:
# list of tables
sql_table.keys()

dict_keys(['disease', 'patients', 'encounters', 'careplans', 'conditions', 'medications', 'procedures', 'observations', 'devices', 'imaging_studies', 'immunizations'])

In [40]:
 pd.read_csv(f'/content/drive/MyDrive/csv_data_Oliver/covid19/encounters.csv').head()

,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
0,7627d701-6c75-be69-0fcc-aeb32ea27270,2013-10-21T09:22:47Z,2013-10-21T09:37:47Z,f7dddcba-380e-7fd4-20b0-da086ce5dc3a,3feeb8f0-7e8a-3cbb-8557-3cd7df027ce1,7f540f46-bac2-342c-bc19-49f3689f92fa,42c4fca7-f8a9-3cd1-982a-dd9751bf3e2a,wellness,162673000,General examination of patient (procedure),129.16,129.16,129.16,NaN,NaN
1,e0361112-89b9-9057-f577-ce5878f47f86,2011-09-17T21:57:21Z,2011-09-17T22:12:21Z,ad5525c0-fdab-94dc-018f-d7209c72bdef,1248b93c-56ef-3c3a-adab-56eb2c991c4d,815da5a3-1099-3657-ab9e-68d4e6c601cc,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,162673000,General examination of patient (procedure),129.16,129.16,129.16,NaN,NaN
2,2891a0e4-ee92-6a31-c4e4-5de8cedddf19,2011-09-11T17:40:13Z,2011-09-11T17:55:13Z,2812f4fc-6aba-0033-728b-7e569bb8adcc,5b1e6842-ca09-3521-ae67-3dbf4ff7638f,03912e14-e103-393a-9c30-9c7dc13938f9,7caa7254-5050-3b5e-9eae-bd5ea30e809c,wellness,162673000,General examination of patient (procedure),129.16,129.16,129.16,NaN,NaN
3,a736e521-64e7-fe9c-6261-8776b3ab3d76,2012-02-24T15:16:17Z,2012-02-24T15:31:17Z,7f30313f-98e9-2cc2-68a5-6f9973f9ad5a,5a9ded14-4f2b-3d66-9301-02c8fb81c8fd,b4a12dd1-91cf-3c2a-a120-4390255fda1c,047f6ec3-6215-35eb-9608-f9dda363a44c,wellness,162673000,General examination of patient (procedure),129.16,129.16,129.16,NaN,NaN
4,1f7153da-bb06-0645-7e15-1f82cff08975,2014-12-12T15:16:17Z,2014-12-12T15:31:17Z,7f30313f-98e9-2cc2-68a5-6f9973f9ad5a,5a9ded14-4f2b-3d66-9301-02c8fb81c8fd,b4a12dd1-91cf-3c2a-a120-4390255fda1c,047f6ec3-6215-35eb-9608-f9dda363a44c,wellness,162673000,General examination of patient (procedure),129.16,129.16,129.16,NaN,NaN


# Indexing

In [39]:
# Indexing for city in the patients table
idx_patients_city = pd.read_sql_query("CREATE INDEX city_index ON patients (City);", conn)

DatabaseError: ignored

# Eingrenzung 

In [41]:
# Look for patients undergoing "Electrical cardioversion" in the procedures table
count_patients_cardioversion = pd.read_sql_query("SELECT COUNT(Description) FROM procedures WHERE Description='Electrical cardioversion';", conn) #Anzahl an Patienten mit Cardioversion
count_patients_cardioversion

,COUNT(Description)
0,182


In [44]:
pat_tables = pd.read_sql_query(
    "CREATE TABLE Persons (
      ID int NOT NULL PRIMARY KEY,
      LastName varchar(255) NOT NULL,
      FirstName varchar(255),
      Age int);"), conn)
pat_tables

SyntaxError: ignored